In [8]:

from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0, 
    groq_api_key="groq_api_key", 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the moon's surface on July 21, 1969, as part of the Apollo 11 mission.


In [10]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-31388?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Machine Learning Engineer – AI/ML (Remote Work Option)

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsData & AnalyticsMachine Learning Engineer – AI/ML (Remote Work Option)Be

In [13]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your task is to extract job postings from the provided text
        role: The title or name of the job position.
        experience: The required years or level of experience for the position.
        skills: A list of key skills or technologies required for the job.
        description: A brief summary or description of the job role, including any important responsibilities or requirements.
        Ensure that each JSON object captures the relevant details accurately. If any of the required information is missing or unclear in the text, leave the corresponding key empty or use null.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res)

content='[\n  {\n    "role": "Machine Learning Engineer – AI/ML",\n    "experience": null,\n    "skills": [\n      "Machine Learning",\n      "Python",\n      "SQL",\n      "Scikit-learn",\n      "Pandas",\n      "OpenCV",\n      "NumPy",\n      "TensorFlow",\n      "Databricks",\n      "SageMaker",\n      "Spark",\n      "AWS cloud",\n      "CI/CD pipelines",\n      "containerization",\n      "Generative AI"\n    ],\n    "description": "Develop advanced analytics systems that directly impact the business. Work on a cross-disciplinary team to enable data-driven decision making across multiple organizations. Create high-quality solutions that power Nike, working at the intersection of machine learning and software engineering."\n  }\n]' response_metadata={'token_usage': {'completion_tokens': 163, 'prompt_tokens': 1816, 'total_tokens': 1979, 'completion_time': 0.652, 'prompt_time': 0.45359016, 'queue_time': 0.005739005999999991, 'total_time': 1.10559016}, 'model_name': 'llama-3.1-70b-ver

In [14]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Machine Learning Engineer – AI/ML',
  'experience': None,
  'skills': ['Machine Learning',
   'Python',
   'SQL',
   'Scikit-learn',
   'Pandas',
   'OpenCV',
   'NumPy',
   'TensorFlow',
   'Databricks',
   'SageMaker',
   'Spark',
   'AWS cloud',
   'CI/CD pipelines',
   'containerization',
   'Generative AI'],
  'description': 'Develop advanced analytics systems that directly impact the business. Work on a cross-disciplinary team to enable data-driven decision making across multiple organizations. Create high-quality solutions that power Nike, working at the intersection of machine learning and software engineering.'}]

In [15]:
type(json_res)

list

In [16]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [17]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [24]:
links = collection.query(query_texts=['Experience in Python','Experience in React'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [21]:
job = json_res
job[0]['skills']

['Machine Learning',
 'Python',
 'SQL',
 'Scikit-learn',
 'Pandas',
 'OpenCV',
 'NumPy',
 'TensorFlow',
 'Databricks',
 'SageMaker',
 'Spark',
 'AWS cloud',
 'CI/CD pipelines',
 'containerization',
 'Generative AI']

In [23]:
links = collection.query(query_texts=job[0]['skills'], n_results=1).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'}]]

In [26]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at Optisol. Optisol is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Optisol 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at Optisol. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job[0]), "link_list": links})
print(res.content)


Subject: Expert Machine Learning Solutions for Nike's Advanced Analytics Systems

Dear Hiring Manager,

I came across the Machine Learning Engineer – AI/ML role at Nike and was impressed by the opportunity to develop advanced analytics systems that drive business impact. As a Business Development Executive at Optisol, I'd like to introduce our company as a potential partner in fulfilling your machine learning needs.

At Optisol, we specialize in AI & Software Consulting, empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has extensive experience in developing high-quality solutions that integrate machine learning and software engineering.

Our capabilities align with the skills required for the Machine Learning Engineer role, including:

* Machine Learning: We have expertise in developing and deploying machine learning models using popular libraries like Scikit-learn, Tenso